In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import gc
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit

In [3]:
map_info = pd.read_csv('../input/ubiquantstock/map_info.csv')
map_info = map_info[['Main_Section', 'investment_id']]
print(map_info)
data = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
print(data.shape)
data = pd.merge(data, map_info, how='left')
data['Main_Section'].fillna(0, inplace=True)

      Main_Section  investment_id
0              1.0         2586.0
1              3.0         1674.0
2              2.0          604.0
3              2.0         2959.0
4              3.0         2231.0
...            ...            ...
3500           5.0          233.0
3501           5.0         2073.0
3502           5.0         3618.0
3503           5.0         1534.0
3504           2.0          766.0

[3505 rows x 2 columns]
(3141410, 304)


In [4]:
# fea_col = [f'f_{i}' for i in range(300)]
# print(fea_col)
# c = fea_col[0]
# mean = data[c].mean()
# std = data[c].std()
# choose = (data[c] < (mean + std * 70)) & (data[c] > (mean - std * 70))
# for c in range(1, len(fea_col)):
#     c = fea_col[c]
#     mean = data[c].mean()
#     std = data[c].std()
#     choose = choose & ((data[c] < (mean + std * 70)) & (data[c] > (mean - std * 70)))
# print(choose.astype('int').sum())
# data = data[choose]
# print(data.shape)
# data.to_csv('./del_std70_data.csv', index=False)

In [5]:
# data['target_lag1'] = data.groupby('investment_id')['target'].shift(1)
# data['target_lag2'] = data.groupby('investment_id')['target'].shift(2)
# data['target_lag3'] = data.groupby('investment_id')['target'].shift(3)
# data['target_lag4'] = data.groupby('investment_id')['target'].shift(4)
# data['target_lag5'] = data.groupby('investment_id')['target'].shift(5)
# print(data['target_lag1'].isnull().sum())
# print(data['target_lag2'].isnull().sum())
# print(data['target_lag3'].isnull().sum())
# print(data['target_lag4'].isnull().sum())
# print(data['target_lag5'].isnull().sum())

# data['target_lag1'][data['target_lag1'].isnull()] = data['target']
# data['target_lag2'][data['target_lag2'].isnull()] = data['target']
# data['target_lag3'][data['target_lag3'].isnull()] = data['target']
# data['target_lag4'][data['target_lag4'].isnull()] = data['target']
# data['target_lag5'][data['target_lag5'].isnull()] = data['target']
# print(data['target_lag1'].isnull().sum())
# print(data['target_lag2'].isnull().sum())
# print(data['target_lag3'].isnull().sum())
# print(data['target_lag4'].isnull().sum())
# print(data['target_lag5'].isnull().sum())

In [6]:
# tar_lag1 = data['target_lag1'].values
# tar_lag2 = data['target_lag2'].values
# tar_lag3 = data['target_lag3'].values
# tar_lag4 = data['target_lag4'].values
# tar_lag5 = data['target_lag5'].values

# print(tar_lag1.shape, tar_lag1.dtype)

# np.save('./target_lag1.npy', tar_lag1)
# np.save('./target_lag2.npy', tar_lag2)
# np.save('./target_lag3.npy', tar_lag3)
# np.save('./target_lag4.npy', tar_lag4)
# np.save('./target_lag5.npy', tar_lag5)

In [7]:
# vibration = data.groupby('investment_id')['target'].transform(lambda x: len(x.unique())).values
# scaler = MinMaxScaler(feature_range=(1e-3, 1 + 1e-3))
# vibration = scaler.fit_transform(vibration.reshape(-1, 1))
# vibration = vibration.astype('float32')
# np.save('./target_vibration.npy', vibration)
# print(vibration)
# print(vibration.shape)

In [8]:
time_id = data['time_id'].values
target = data['target'].values
target = np.column_stack((time_id, target))
print(target.shape)
np.save('./train_(timeid)target-f32.npy', target)
target = target[:, 1]

inv_id = data['investment_id'].values
np.save('./inv_id.npy', inv_id)

section = data['Main_Section'].values.astype('int')
print(section[:20], section.dtype)
np.save('./section.npy', section)

(3141410, 2)
[6 2 2 5 5 5 3 5 5 1 5 5 5 5 0 5 5 5 5 5] int64


In [9]:
data = data.drop(columns=['time_id', 'row_id', 'investment_id', 'target', 'Main_Section']).values
print(data.shape)
np.save('./ori_train_data_feature_f16.npy', data.astype('float16'))

scaler = StandardScaler()
data = scaler.fit_transform(data)
np.save('./train_std_data-f32.npy', data)
np.save('./train_std_data.npy', data.astype('float16'))

# scale_param = scaler.get_params()
# print(scale_param)
# with open('./scale_param.pkl', 'wb') as f:
#     pickle.dump(scale_param, f)
    

(3141410, 300)


In [10]:
# def create_record(i):
#     dic = {}
#     dic["feature"] = tf.train.Feature(float_list=tf.train.FloatList(value=list(data[i])))
#     dic["target"] = tf.train.Feature(float_list=tf.train.FloatList(value=[target[i]]))
#     record_bytes = tf.train.Example(features=tf.train.Features(feature=dic)).SerializeToString()
#     return record_bytes

# with tf.io.TFRecordWriter('./train_data_target-f32-all.tfrecords') as file_writer:
#     for i in range(data.shape[0]):
#         file_writer.write(create_record(i))